In [1]:
import pandas as pd

In [2]:
# now that we have the df for twitter sentiment of each day fron 20170101
sentiment_filename = 'dailysentiments.csv'
stock_filename = 'stockprice_raw.csv'
sentiments = pd.read_csv(sentiment_filename, engine='python',index_col='date')
stocks = pd.read_csv(stock_filename, engine = 'python',index_col='Date')
# sentiments = sentiments.drop(columns=sentiments.columns[0])

In [3]:
sentiments

,pos_mean_likes,pos_mean_replies,pos_mean_retweet,pos_sd,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neu_sd,neg_mean_likes,neg_mean_replies,neg_mean_retweet,neg_sd
date,,,,,,,,,,,,
20170101,4.207272,3.828412,3.232838,27.881485,8.028677,8.366872,9.094131,27.128056,0.264051,0.304716,0.173031,13.242646
20170102,6.322558,6.260311,5.908047,29.785364,6.780715,7.166920,7.121932,27.058838,1.182442,0.858484,1.255735,10.061570
20170103,1.233669,1.469026,1.436088,30.586431,4.423724,4.048554,4.352946,19.877297,4.342607,4.482420,4.210966,28.945078
20170104,0.821791,0.382300,0.921218,20.913805,4.941648,2.870712,4.963726,23.088163,0.486562,2.996988,0.365056,24.107241
20170105,2.220672,2.516691,2.811572,23.289303,6.303775,5.975591,5.727157,16.825404,0.566462,0.598628,0.552180,16.756974
...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,0.886511,0.862483,0.781388,36.554188,0.093095,0.110687,0.179754,30.409409,0.020394,0.026830,0.038858,21.325716
20201228,0.055150,0.117387,0.041629,34.792115,0.310698,0.787195,0.255934,30.099057,0.634153,0.095418,0.702437,24.682094
20201229,0.471613,0.463504,0.395673,34.188085,0.441821,0.497922,0.437942,28.073527,0.086566,0.038573,0.166385,24.993177


In [4]:
# set the size of the sliding window of stock price prediction
window_size = 1
# use Open_prev to find the opening price of the previous day
stocks['Open_prev'] = stocks['Open'].shift(window_size)

In [5]:
from datetime import timedelta
dates = stocks.index
dates = pd.to_datetime(dates)

#to correct for if the gap is more than one day
for k in range(stocks.shape[0]):
  if k > 0 and (dates[k] - dates[k-1]) > timedelta(days =1):
    idx = stocks.index[k]
    stocks.at[idx, 'Open_prev'] = stocks.iloc[k-1]['Open']

In [6]:
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_prev
Date,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,43.259998
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.972000
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,42.950001
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.284000
...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,642.989990
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,674.510010
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,661.000000


In [ ]:
#add in features of previous two days' stock prices
stocks['prev2_Diff'] = stocks.shift(2)["Close"] - stocks.shift(3)["Open"]
stocks.head(10)

In [7]:
stocks['Diff'] = stocks['Close'] - stocks['Open_prev']
stocks.loc[stocks['Diff']> 0, 'Diff'] = 1
stocks.loc[stocks['Diff']< 0, 'Diff'] = -1
stocks.loc[stocks['Diff'] == 0, 'Diff'] = 0
stock_label_df = stocks[['date','Diff']]
# stock_label_df = stocks[['date','prev2_Diff','Diff']]

In [8]:
stock_label_df
#stock_label_df.to_csv('stockprice_label.csv')

,date,Diff
Date,,
2016-12-30,20161230,NaN
2017-01-03,20170103,1.0
2017-01-04,20170104,1.0
2017-01-05,20170105,1.0
2017-01-06,20170106,1.0
...,...,...
2020-12-28,20201228,1.0
2020-12-29,20201229,-1.0
2020-12-30,20201230,1.0


In [21]:
'''
previous code:
# Create a df with sentiment analysis from previous two days given the dateindex 
sentiment_prev2days = pd.concat([sentiments.shift(1),sentiments.shift(2)],axis=1)
sentiment_prev2days = sentiment_prev2days.dropna()
sentiment_prev2days.insert(0,'date', sentiment_prev2days.index)
sentiment_prev2days.index.names = ['Date']
sentiment_prev2days
# note that 1448 tweet days vs. 1007 valid stock labels

'''

#sentiment analysis window
sentiment_window_size = 10
#features = ["pos_mean", "pos_sd", "neu_mean", "neu_sd", "neg_mean", "neg_sd"]
#features = ["pos_mean_likes", "pos_mean_replies", "pos_mean_retweet","neu_mean_likes", "neu_mean_replies", "neu_mean_retweet","neg_mean_likes", "neg_mean_replies", "neg_mean_retweet"]
features = ["pos_mean_likes", "pos_mean_replies", "pos_mean_retweet","pos_sd","neu_mean_likes", "neu_mean_replies", "neu_mean_retweet","neu_sd","neg_mean_likes", "neg_mean_replies", "neg_mean_retweet","neg_sd"]
day_features = [("day{}_"+s).format(sentiment_window_size) for s in features]
#first shift to the day before stock price window
sentiment_shift = sentiments.shift(window_size+1)
sentiment_shift.columns = day_features

k = sentiment_window_size - 1

#accumulate features of sentiment analysis window
for i in range(3, sentiment_window_size + 2):
  day_features = [("day{}_"+s).format(k) for s in features]
  df = sentiments.shift(i)
  df.columns = day_features
  sentiment_shift = pd.concat([sentiment_shift,df],axis=1)
  k -= 1

# #accumulate features of sentiment analysis window
# for i in range(3, sentiment_window_size + 2):
#   sentiment_shift = pd.concat([sentiment_shift,sentiments.shift(i)],axis=1)

sentiment_shift = sentiment_shift.dropna()

sentiment_shift

,day10_pos_mean_likes,day10_pos_mean_replies,day10_pos_mean_retweet,day10_pos_sd,day10_neu_mean_likes,day10_neu_mean_replies,day10_neu_mean_retweet,day10_neu_sd,day10_neg_mean_likes,day10_neg_mean_replies,...,day1_pos_mean_retweet,day1_pos_sd,day1_neu_mean_likes,day1_neu_mean_replies,day1_neu_mean_retweet,day1_neu_sd,day1_neg_mean_likes,day1_neg_mean_replies,day1_neg_mean_retweet,day1_neg_sd
date,,,,,,,,,,,,,,,,,,,,,
20170112,3.540877,2.980973,3.052879,24.444988,2.571007,2.997780,3.021332,25.364667,0.138117,0.271247,...,3.232838,27.881485,8.028677,8.366872,9.094131,27.128056,0.264051,0.304716,0.173031,13.242646
20170113,2.688410,2.520085,2.844737,17.050466,5.373914,5.619566,5.162004,14.922122,0.271010,0.193683,...,5.908047,29.785364,6.780715,7.166920,7.121932,27.058838,1.182442,0.858484,1.255735,10.061570
20170114,3.733475,4.148451,3.406968,33.489675,5.166724,4.770145,5.551680,32.850840,0.190710,0.172314,...,1.436088,30.586431,4.423724,4.048554,4.352946,19.877297,4.342607,4.482420,4.210966,28.945078
20170115,3.328424,3.373077,3.273070,29.479468,3.762797,3.718755,3.818115,28.677300,0.051636,0.051025,...,0.921218,20.913805,4.941648,2.870712,4.963726,23.088163,0.486562,2.996988,0.365056,24.107241
20170116,6.267532,5.798808,3.531834,29.623654,4.044157,3.411901,3.911920,28.237919,6.354978,7.455957,...,2.811572,23.289303,6.303775,5.975591,5.727157,16.825404,0.566462,0.598628,0.552180,16.756974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,0.233273,0.461738,0.186157,36.785228,0.448576,0.485939,0.455544,30.526596,0.318151,0.052323,...,0.134903,32.760017,0.634783,0.643418,0.779769,29.297728,0.117921,0.124484,0.085328,22.240346
20201228,0.316511,0.409077,0.364218,35.428170,0.397077,0.424900,0.539451,32.545976,0.286412,0.166023,...,0.320990,35.763949,0.434796,0.511585,0.580117,31.761274,0.113299,0.127609,0.098894,25.758550
20201229,0.886511,0.862483,0.781388,36.554188,0.093095,0.110687,0.179754,30.409409,0.020394,0.026830,...,0.349909,37.263068,0.460328,0.439043,0.490845,31.531100,0.172524,0.246647,0.159246,19.530237


In [22]:
dataframe = sentiment_shift.merge(stock_label_df,how='left', left_on='date', right_on='date')
dataframe = dataframe.dropna()
dataframe

,date,day10_pos_mean_likes,day10_pos_mean_replies,day10_pos_mean_retweet,day10_pos_sd,day10_neu_mean_likes,day10_neu_mean_replies,day10_neu_mean_retweet,day10_neu_sd,day10_neg_mean_likes,...,day1_pos_sd,day1_neu_mean_likes,day1_neu_mean_replies,day1_neu_mean_retweet,day1_neu_sd,day1_neg_mean_likes,day1_neg_mean_replies,day1_neg_mean_retweet,day1_neg_sd,Diff
0,20170112,3.540877,2.980973,3.052879,24.444988,2.571007,2.997780,3.021332,25.364667,0.138117,...,27.881485,8.028677,8.366872,9.094131,27.128056,0.264051,0.304716,0.173031,13.242646,1.0
1,20170113,2.688410,2.520085,2.844737,17.050466,5.373914,5.619566,5.162004,14.922122,0.271010,...,29.785364,6.780715,7.166920,7.121932,27.058838,1.182442,0.858484,1.255735,10.061570,1.0
5,20170117,2.118684,2.939366,1.630829,21.786983,5.749531,4.972113,6.477375,25.828349,0.465118,...,22.936842,3.746757,4.294002,3.749259,23.416791,0.656144,0.207291,0.216844,23.076308,1.0
6,20170118,2.277675,3.061165,2.844041,6.762477,11.918982,14.455333,14.616036,20.916211,5.803344,...,21.676647,2.431541,2.477603,2.434837,20.831707,0.209339,0.196020,0.192863,11.728336,1.0
7,20170119,9.642129,8.771922,9.863965,19.008594,6.897518,7.731511,6.674546,18.196428,0.127020,...,32.156001,4.156664,5.277351,4.300867,31.538552,0.074348,0.082443,0.062210,1.290825,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1407,20201224,0.072582,0.075541,0.060729,26.726447,0.404326,0.423167,0.430870,28.612751,0.523092,...,33.436777,0.696308,0.760189,0.757598,30.240124,0.063233,0.056428,0.051044,22.706860,1.0
1411,20201228,0.316511,0.409077,0.364218,35.428170,0.397077,0.424900,0.539451,32.545976,0.286412,...,35.763949,0.434796,0.511585,0.580117,31.761274,0.113299,0.127609,0.098894,25.758550,1.0
1412,20201229,0.886511,0.862483,0.781388,36.554188,0.093095,0.110687,0.179754,30.409409,0.020394,...,37.263068,0.460328,0.439043,0.490845,31.531100,0.172524,0.246647,0.159246,19.530237,-1.0
1413,20201230,0.055150,0.117387,0.041629,34.792115,0.310698,0.787195,0.255934,30.099057,0.634153,...,34.693774,0.397160,0.551848,0.416572,27.764325,0.285088,0.150445,0.200466,25.259589,1.0


In [23]:

filename = "database_prev{}days_attemp3.csv".format(sentiment_window_size)
filename

'database_prev10days_attemp3.csv'

In [24]:
dataframe.to_csv(filename)